# Experiment 03: Z3 Runtime Constraint Enforcement

## Goal

Use Z3 at **runtime** to enforce formally verified constraints from our ACL2 specification. This addresses the limitation discovered in Experiment 02: SMTLink only generates Python when Z3 is actually invoked during proof (our theorems were simple enough that ACL2 proved them without Z3).

## Key Insight

Instead of relying on SMTLink's proof-time Python generation, we:
1. Keep ACL2 as the **formal specification** (proven correct)
2. **Manually translate** the ACL2 spec to Z3 Python (following SMTLink's patterns)
3. Use Z3 at **runtime** for constraint enforcement and optimization

## Files Created

| File | Purpose |
|------|---------|
| [z3_react_constraints.py](z3_react_constraints.py) | Z3 constraint module mirroring ACL2 spec |
| [experiment-03-z3-runtime.ipynb](experiment-03-z3-runtime.ipynb) | Demo notebook with tests |

## Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                     Formal Specification                        │
│  experiment-01-react-verified.lisp (ACL2 - proven correct)     │
└─────────────────────────────────────────────────────────────────┘
                              │
                              │ manual translation (following SMTLink patterns)
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│                  z3_react_constraints.py                        │
│  - Z3 Datatypes (AgentState, ToolSpec)                         │
│  - Constraint functions (can_invoke_tool, must_respond, etc.)  │
│  - Runtime checker (ReactConstraintChecker)                    │
│  - Optimizer (optimize_tool_selection)                         │
└─────────────────────────────────────────────────────────────────┘
                              │
                              │ enforces at runtime
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│                    LangGraph Agent Loop                         │
│  - Before tool call: check_tool_invocation()                   │
│  - After LLM response: check_continue_decision()               │
│  - Tool selection: optimize_tool_selection()                   │
└─────────────────────────────────────────────────────────────────┘
```

## ACL2 → Z3 Python Correspondence

| ACL2 (experiment-01) | Python (z3_react_constraints.py) |
|----------------------|----------------------------------|
| `(defprod agent-state ...)` | `AgentState = Datatype(...)` |
| `(defprod tool-spec ...)` | `ToolSpec = Datatype(...)` |
| `(defun access-sufficient-p ...)` | `def access_sufficient(...)` |
| `(defun tool-permitted-p ...)` | `def tool_permitted(...)` |
| `(defun can-invoke-tool-p ...)` | `def can_invoke_tool(...)` |
| `(defun must-respond-p ...)` | `def must_respond(...)` |
| `(defun should-continue-p ...)` | `def should_continue(...)` |
| `(defun deduct-tool-cost ...)` | `def deduct_tool_cost(...)` |

## Key Results

| Test | Result |
|------|--------|
| Permission constraints | ✅ read_file/web_search allowed, write_file/run_python blocked |
| Continue/respond decision | ✅ Correctly identifies all 5 scenarios |
| Z3 Optimization (full perms) | ✅ Selects optimal tool per objective |
| Z3 Optimization (limited perms) | ✅ Excludes forbidden tools |
| Agent loop simulation | ✅ 7 iterations until budget exhausted |
| ACL2 theorem correspondence | ✅ `permission-safety` and `termination-by-iteration` verified |

## Lessons Learned

1. **SMTLink is proof-time, not runtime**: SMTLink generates Z3 Python only when the SMT solver is actually needed during proof. Simple theorems are proved by ACL2's clause processors before reaching Z3.

2. **Manual translation is straightforward**: The mapping from ACL2 to Z3 Python follows clear patterns:
   - `defprod` → `Datatype`
   - `defun` returning boolean → function returning `BoolRef`
   - `and/or/not/implies` → `And/Or/Not/Implies`
   - `<=/>=/</>` → direct comparison operators

3. **Z3 Optimize enables runtime optimization**: Beyond just checking constraints, Z3's `Optimize` solver can find the best permitted tool given multiple objectives (min cost, time, tokens).

4. **Theorem verification transfers**: The ACL2-proven properties (permission-safety, termination-by-iteration) can be independently verified by Z3, confirming the translation is correct.

## Next Steps (Experiment 04)

- Integrate `z3_react_constraints.py` with LangGraph
- Add LLM-as-judge for satisfaction scoring  
- Connect to LM Studio for local model inference
- Full end-to-end verified agent execution